# Part 1: Data Preparation

Please make sure you have __notebook__ and __nltk__ Python packages installed in the compute context you choose as kernel. For demonstration purpose, this series of notebooks uses the `local` compute context.

**NOTE**: Python 3 kernel doesn't include Azure Machine Learning Workbench functionalities. Please switch the kernel to `local` before continuing further. 

To install __notebook__ and __nltk__, please uncomment and run the following script.

In [1]:
# !pip install --upgrade notebook
# !pip install --upgrade nltk

### Import Required Python Modules

In [2]:
import pandas as pd
import numpy as np
import re, os, gzip, requests, warnings
warnings.filterwarnings("ignore")

## Access Sample Data

In this example, we have collected a set of Q&A pairs from Stack Overflow site tagged as `JavaScript` questions. The data contains 1,201 original Q&A pairs as well as many duplicate questions, i.e. new questions that Stack Overflow users have linked back to pre-existing Q&A pairs that effectively provide answers to these new questions. The data schema of the original questions (Q), duplicate questions (D), and answers (A) can be found in the following table:

| Dataset | Field | Type | Description
| ----------|------------|------------|--------
| question (Q) | Id | String | The unique question ID (primary key)
|  | AnswerId | String | The unique answer ID per question
|  | Text0 | String | The raw text data including the question's title and body
|  | CreationDate | Timestamp | The timestamp of when the question has been asked
| dupes (D) | Id | String | The unique duplication ID (primary key)
|  | AnswerId | String | The answer ID associated with the duplication
|  | Text0 | String | The raw text data including the duplication's title and body
|  | CreationDate | Timestamp | The timestamp of when the duplication has been asked
| answers (A) | Id | String | The unique answer ID (primary key)
|  | text0 | String | The raw text data of the answer

The datasets are compressed and stored in Azure Blob storage as `.tsv.gz` files and this section provides you the code to retreive the data in the notebook.

In [3]:
# load raw data from a .tsv.gz file into Pandas data frame.
def read_csv_gz(url, **kwargs):
    df = pd.read_csv(gzip.open(requests.get(url, stream=True).raw, mode='rb'), sep='\t', encoding='utf8', **kwargs)
    return df.set_index('Id')

In [4]:
# URLs to Original questions, Duplications, and Answers.
questions_url = 'https://bostondata.blob.core.windows.net/stackoverflow/orig-q.tsv.gz'
dupes_url = 'https://bostondata.blob.core.windows.net/stackoverflow/dup-q.tsv.gz'
answers_url = 'https://bostondata.blob.core.windows.net/stackoverflow/ans.tsv.gz'

# load datasets.
questions = read_csv_gz(questions_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
dupes = read_csv_gz(dupes_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
answers = read_csv_gz(answers_url, names=('Id', 'Text0'))

To provide some example, here are the first five rows of the __questions__ table:

In [5]:
questions.head(5)

,AnswerId,Text0,CreationDate
Id,,,
220231,220233,Accessing the web page's HTTP Headers in JavaS...,2008-10-20 22:54:38.767
391979,810461,Get client IP using just JavaScript?. <p>I nee...,2008-12-24 18:22:30.780
109086,109091,Stop setInterval call in JavaScript. <p>I am u...,2008-09-20 19:29:55.377
46155,46181,Validate email address in JavaScript?. <p>How ...,2008-09-05 16:10:11.093
121499,121708,"When onblur occurs, how can I find out which e...",2008-09-23 14:48:43.483


Here is the full text of one __original__ question, whose is `Id` is `220231`. The `AnswerId` associated with this question is `220233`.

In [6]:
# This text include the HTML code.
print(questions["Text0"][220231])

Accessing the web page's HTTP Headers in JavaScript. <p>How do I access a page's HTTP response headers via JavaScript?</p> <p>Related to <a href="http://stackoverflow.com/questions/220149/how-do-i-access-the-http-request-header-fields-via-javascript"><strong>this question</strong></a>, which was modified to ask about accessing two specific HTTP headers.</p> <blockquote> <p><strong>Related:</strong><br> <a href="http://stackoverflow.com/questions/220149/how-do-i-access-the-http-request-header-fields-via-javascript">How do I access the HTTP request header fields via JavaScript?</a></p> </blockquote>


Here is the full text of the __answer__ associated with the above original question:

In [7]:
print(answers["Text0"][220233])

<p>Unfortunately, there isn't an API to give you the HTTP response headers for your initial page request. That was the original question posted here. It has been <a href="http://stackoverflow.com/questions/12258705/how-can-i-read-the-current-headers-without-making-a-new-request-with-js">repeatedly asked</a>, too, because some people would like to get the actual response headers of the original page request without issuing another one.</p> <h1><br/>For AJAX Requests:</h1> <p>If an HTTP request is made over AJAX, it is possible to get the response headers with the <strong><code>getAllResponseHeaders()</code></strong> method. It's part of the XMLHttpRequest API. To see how this can be applied, check out the <em><code>fetchSimilarHeaders()</code></em> function below. Note that this is a work-around to the problem that won't be reliable for some applications.</p> <pre><code>myXMLHttpRequest.getAllResponseHeaders(); </code></pre> <ul> <li><p>The API was specified in the following candidate r

__Duplicate__ questions share the same `AnswerId` as the original question they link to. Here is the first duplicate question linked to the above original question:

In [8]:
print(dupes.query("AnswerId == 220233").iloc[0]["Text0"])

Monitoring http request header on a page. <blockquote> <p><strong>Possible Duplicates:</strong><br> <a href="http://stackoverflow.com/questions/220231/accessing-http-headers-in-javascript">Accessing HTTP Headers in Javascript?</a><br> <a href="http://stackoverflow.com/questions/220149/how-do-i-access-the-http-request-header-fields-via-javascript">How do I access the HTTP request header fields via JavaScript?</a> </p> </blockquote> <p>We can use httpwatch on IE or httpfox on Firefox to monitor http activity</p> <p>If i don't want to use any plugs on browser...</p> <p>Is it possible to monitor http request header on a page just by javascript?</p>


## Pre-process Text Data

### Clean up text

The raw data is in `HTML` format and needs to be cleaned up for any further analysis. We exclude HTML tags, links and code snippets from the data.

In [9]:
# remove embedded code chunks, HTML tags and links/URLs.
def clean_text(text):
    global EMPTY
    EMPTY = ''
    
    if not isinstance(text, str): 
        return text
    text = re.sub('<pre><code>.*?</code></pre>', EMPTY, text)

    def replace_link(match):
        return EMPTY if re.match('[a-z]+://', match.group(1)) else match.group(1)
    
    text = re.sub('<a[^>]+>(.*)</a>', replace_link, text)
    return re.sub('<[^>]+>', EMPTY, text)

In [10]:
for df in (questions, dupes, answers):
    df['Text'] = df['Text0'].apply(clean_text).str.lower()
    df['NumChars'] = df['Text'].str.len()

### Set data selection criteria

To obtain the high quality datasets for phrase learning and model training, we requires a minimum length of characters in the text field. Different thresholds are considered for original questions, duplications, and answers, respectively. Also, each Q&A pair in our set must have a minimum of 3 additional semantically equivalent duplicate questions linked to it. 

In [11]:
# find the AnswerIds has at least 3 dupes.
def find_answerId(answersC, dupesC, num_dupes):
       
    countHash = {}
    for i in dupesC.AnswerId:
        if i not in answersC.index.values:
            continue
        if i not in countHash.keys():
            countHash[i] = 1
        else:
            countHash[i] += 1
            
    countHash = {k: v for k, v in countHash.items() if v >= num_dupes}
    commonAnswerId = countHash.keys()
    
    return commonAnswerId

# extract data based on the selection criteria.
def select_data(questions, dupes, answers):
    # exclude the records without any text
    questions_nz = questions.query('NumChars > 0')
    dupes_nz = dupes.query('NumChars > 0')
    answers_nz = answers.query('NumChars > 0')

    # get the 10th percentile of text length as the minimum length of characters to consider in the text field
    minLenQ = questions_nz.quantile(.1)['NumChars']
    minLenD = dupes_nz.quantile(.1)['NumChars']
    minLenA = answers_nz.quantile(.1)['NumChars']
    
    # eliminate records with text less than the minimum length
    questionsC = questions.query('NumChars >' + str(int(minLenQ)))
    dupesC = dupes.query('NumChars >' + str(minLenD))
    answersC = answers.query('NumChars >' + str(minLenA))
    
    # remove the records in dupesC whose questionId has already existed in questionsC
    duplicatedIndex = list(set(questionsC.index).intersection(set(dupesC.index)))
    dupesC.drop(duplicatedIndex, inplace=True)
    
    # make sure Questions 1:1 match with Answers 
    matches = questionsC.merge(answersC, left_on = 'AnswerId', right_index = True)
    questionsC = matches[['AnswerId', 'Text0_x', 'CreationDate', 'Text_x', 'NumChars_x']]
    questionsC.columns = ['AnswerId', 'Text0', 'CreationDate', 'Text', 'NumChars']

    answersC = matches[['Text0_y', 'Text_y', 'NumChars_y']]
    answersC.index = matches['AnswerId']
    answersC.columns = ['Text0', 'Text', 'NumChars']
    
    # find the AnswerIds has at least 3 dupes
    commonAnswerId = find_answerId(answersC, dupesC, 3)
    
    # select the records with those AnswerIds
    questionsC = questionsC.loc[questionsC.AnswerId.isin(commonAnswerId)]
    dupesC = dupesC.loc[dupesC.AnswerId.isin(commonAnswerId)]
    
    return questionsC, dupesC

In [12]:
# some questions have been linked to multiple AnswerIds.
# we keep the first AnswerId associated with that question and remove the rest.
questions = questions.groupby(questions.index).first()
dupes = dupes.groupby(dupes.index).first()

# execute the data selection function on questions, dupes and answers.
questionsC, dupesC = select_data(questions, dupes, answers)

## Prepare Training and Test datasets

In this example, we retain original question and 75% of the duplicate questions for training, and hold-out the most recently posted 25% of duplicate questions as test data. The training and test data are split by `CreationDate`.

- training set = Original questions + 75% of oldest Duplications per original question
- test set = remaining 25% of Duplications per original question

In [13]:
# split Original questions and their Duplications into training and test sets.
def split_data(questions, dupes, frac):
    trainQ = questions
    testQ = pd.DataFrame(columns = dupes.columns.values) # create an empty data frame

    for answerId in np.unique(dupes.AnswerId):
        df = dupes.query('AnswerId == ' + str(answerId))
        totalCount = len(df)
        splitPoint = int(totalCount * frac)
        dfSort = df.sort_values(by = ['CreationDate'])
        trainQ = trainQ.append(dfSort.head(splitPoint)) # oldest N percent of duplications
        testQ = testQ.append(dfSort.tail(totalCount - splitPoint))

    # convert data type to int
    testQ[["AnswerId", "NumChars"]] = testQ[["AnswerId", "NumChars"]].astype(int) 
    # rename the index 
    testQ.index.rename("Id", inplace=True)
    
    return trainQ, testQ

In [14]:
trainQ, testQ = split_data(questionsC, dupesC, 0.75)

In [15]:
trainQ.head(5)

,AnswerId,Text0,CreationDate,Text,NumChars
Id,,,,,
3713,3777,Call ASP.NET Function From Javascript?. <p>I'm...,2008-08-06 17:16:36.630,call asp.net function from javascript?. i'm wr...,227
5223,6700,"Length of a JavaScript object (that is, associ...",2008-08-07 19:42:21.060,"length of a javascript object (that is, associ...",313
7477,7523,Autosizing textarea using Prototype. <p>I'm cu...,2008-08-11 01:43:13.493,autosizing textarea using prototype. i'm curre...,1664
18082,1830844,Validate decimal numbers in JavaScript - IsNum...,2008-08-20 14:21:13.793,validate decimal numbers in javascript - isnum...,231
21294,242607,Dynamically load a JavaScript file. <p>How can...,2008-08-21 21:59:31.080,dynamically load a javascript file. how can yo...,980


## Select Subsets with Sufficient Training Questions per Answer Class

In our past experiments, we notice that some Q&A pairs only link to a small number of duplicate questions. This means those answer classes may contain an insufficient amount of examples to train an accurate model. We examine the effect of the number of duplicate questions available for training for each Q&A pair. 

<img src="https://raw.githubusercontent.com/Azure/MachineLearningSamples-QnAMatching/master/Image/training_size.PNG?token=APoO9rnKXamwVdXu8luA_Dd28UUBncwrks5ZwtRowA%3D%3D">

The above Figure shows results for questions relative to the number of training examples available for the correct Q&A pair that should be returned. Most of our Q&A pairs (857 out of 1201) have 5 or fewer known duplicate questions available for training. Performance on these questions is relatively weak, with the correct Q&A pair landing in the top 10 results less than 40% of the time. However, when greater numbers of duplicate questions are available for training, performance improves dramatically; when Q&A pairs have 50 or more duplicate questions available for training, the classification model places these pairs in the top 10 of the retrieved results 98% of the time when they correctly match the query. The most duplicated question contains 962 duplications. 

For the study in this notebook, we only consider the answer classes that have more than 13 training questions (original and duplicate questions) in this notebook. This reduces the entire dataset to 5,153 training questions, 1,735 test questions, and 103 unique answer classes.

In [16]:
countPerAns = pd.DataFrame({"NumTrain" : trainQ.groupby("AnswerId").size()})
trainQwithCount = trainQ.merge(countPerAns, left_on="AnswerId", right_index=True)
testQwithCount = testQ.merge(countPerAns, left_on="AnswerId", right_index=True)

# for each Answer class, we request more than 13 training questions.
trainQ = trainQwithCount[trainQwithCount["NumTrain"] > 13]
testQ = testQwithCount[testQwithCount["NumTrain"] > 13]

In [17]:
print("# of training examples: " + str(len(trainQ)))
print("# of testing examples: " + str(len(testQ)) + "\n")
print("A quick glance of the training data: \n")
trainQ[["AnswerId", "Text"]].head(5)

# of training examples: 5153
# of testing examples: 1735

A quick glance of the training data: 



,AnswerId,Text
Id,,
69913,69984,why don't self-closing script tags work?. what...
392561,69984,firefox script tag error. while adding some ve...
1297308,69984,weird javascript/jquery behavior. possible du...
3352182,69984,html: why script tags should always have full ...
5355867,69984,loading scripts in javascript. possible dupli...


## Save Outputs to a Share Directory in the Workbench

Azure Machine Learning Workbench provides a flexible way of saving intermediate files. `os.environ.get('AZUREML_NATIVE_SHARE_DIRECTORY')` retrieves a share directory where the files are stored. Those files can be accessed from other notebooks or Python files.

In [18]:
workfolder = os.environ.get('AZUREML_NATIVE_SHARE_DIRECTORY')
trainQ.to_csv(os.path.join(workfolder, 'trainQ_part1'), sep='\t', header=True, index=True, index_label='Id')
testQ.to_csv(os.path.join(workfolder, 'testQ_part1'), sep='\t', header=True, index=True, index_label='Id')